## Compare results for multiple results for 512 square datasets 
- Aug 11, 2020 : 512 square dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1

def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=int(img_size/2)
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_err'][:idx]-dict_sample['spec_err'][:idx])**2))
        print(spec_loss)
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict

In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[5000:8000][:,0,:,:]
print(s_val.shape)
img_size=s_val.shape[1]

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(3000, 512, 512)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:2000][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


29.10128736198687


In [9]:
# ### Load keras images
# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
# lst=['gen_imgs_1.npy','gen_imgs_2.npy']
# for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
#     key='keras_'+strg
#     fname=parent_dir+'run2_512_square_20epochs/models/'+fldr
#     dict1={'label':key,'fname':fname}

#     images=np.load(dict1['fname'])[:2000]     ### No transform needed here
# #     print(np.max(images),np.min(images))

#     dict1['num_img']=images.shape[0]
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)


In [10]:
# # ### Extract a few images generated by Lbann
# parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/'

# lst=[]
# fldr='20200905_080143_bsize10_test/gen_imgs_chkpt/epoch.36.step.81000_20200908_121732/'
# lst.append(fldr)
# fldr='20200905_080143_bsize20_test/gen_imgs_chkpt/epoch.36.step.40500_20200908_124523/'
# lst.append(fldr)
# fldr='20200905_080143_bsize25_test/gen_imgs_chkpt/epoch.34.step.30600_20200908_124721/'
# lst.append(fldr)
# fldr='20200905_080143_bsize50_test/gen_imgs_chkpt/epoch.38.step.17100_20200908_124321/'
# lst.append(fldr)
# fldr='20200904_200005_bsize100_test/gen_imgs_chkpt/epoch.36.step.8100_20200908_123928/'
# lst.append(fldr)

# # lst=['20200806_070111_gen_img_exagan/']
# names=['1_b10','2_b20','3_b25','4_b50','5_b100']

# for strg,fldr in zip(names,lst):
#     f_strg=parent_dir+fldr+'dump_outs/trainer0/model0/sgd.testing.epoch.*.step.*_gen_img_instance1_activation_output0.npy'
#     f_list=glob.glob(f_strg)
# #     print(f_list)
#     key='lbann_'+strg
# #     fname=parent_dir+fldr+'/models/gen_imgs.npy'
#     dict1={'label':key,'fname':f_strg}
#     arr=[np.load(fname)[:,0,:,:] for fname in f_list]
#     images=np.vstack(arr)
#     print(images.shape)
#     dict1['num_img']=images.shape[0]
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)



In [11]:
### Extract some LBANN training run data

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20201112_063545_bsize64_test_512/dump_outs/trainer0/model0/'

lst=[(10,3870),(10,3770),(15,5560),(15,5280)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key='lbann_train_{0}-{1}'.format(epoch,step)
    fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_img']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

# print(s_lbann.shape,np.max(s_lbann))



{'label': 'lbann_train_10-3870', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20201112_063545_bsize64_test_512/dump_outs/trainer0/model0/sgd.training.epoch.10.step.3870_gen_img_instance1_activation_output0.npy', 'num_img': 64}
41.263383601394736
{'label': 'lbann_train_10-3770', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20201112_063545_bsize64_test_512/dump_outs/trainer0/model0/sgd.training.epoch.10.step.3770_gen_img_instance1_activation_output0.npy', 'num_img': 64}
42.18878283073826
{'label': 'lbann_train_15-5560', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20201112_063545_bsize64_test_512/dump_outs/trainer0/model0/sgd.training.epoch.15.step.5560_gen_img_instance1_activation_output0.npy', 'num_img': 64}
41.90095278470231
{'label': 'lbann_train_15-5280', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20201112_063545_bsize64_test_512/dump_outs/trainer0/model0/sgd.training.epoch.15.s

In [12]:
print(df_runs.columns)
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')


In [13]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0003905041503886839, 0.00028129094188102636...","[1.1193483149075099, 0.9587987861501517, 0.258...","[24994.125958279594, 14433.198206567558, 12220...","[2620431.409475068, 2298491.0857654694, 205209...",0.000057,0.000004,0.000007,0.000046,6.596392e-07,1.252274,0.001517,433.036406,2000.0
1,lbann_train_10-3870,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.005605278223036637, 0.0035878049222682184, ...","[1.1326528664299016, 0.9271217944356364, 0.271...","[100493.53376099785, 92880.1426094659, 64916.4...","[1618357.695323944, 1657703.609022331, 1548103...",0.167840,0.003997,0.024563,0.139280,1.763820e-03,15.213772,18.752943,135542.239971,64.0
2,lbann_train_10-3770,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.004984088910429684, 0.002910991957719613, 0...","[1.0429222928229844, 1.069789142091788, 0.2906...","[98742.47083585044, 70173.64348246186, 76664.5...","[1334243.0403518677, 1259718.5585098267, 12489...",0.142617,0.039557,0.086314,0.016745,2.109425e-02,12.176125,34.187801,359917.273743,64.0
3,lbann_train_15-5560,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0038799804612700555, 0.002828741475423181, ...","[1.0873457602898673, 1.0134039587509438, 0.271...","[85086.15945189084, 86620.68031344339, 65735.9...","[1300961.052009344, 1386831.02668705, 1220048....",0.389942,0.014280,0.045485,0.330177,5.161448e-03,10.985656,11.024848,94855.230201,64.0
4,lbann_train_15-5280,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003044185094524321, 0.002982016168203042, 0...","[1.1077670856779653, 0.975202024440215, 0.2710...","[92819.28434024566, 73284.00335892697, 79697.7...","[1552872.5898694992, 1590209.0040247198, 15166...",0.216982,0.006490,0.043560,0.166933,1.311232e-03,8.360818,14.903844,235623.709541,64.0


### Compare chi-squares

In [ ]:
chi_keys=['label','chi_1','chi_spec1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'num_img','fname']
df_runs[chi_keys]

In [ ]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [ ]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

In [ ]:
col_list=['label','chi_1','chi_spec1','num_img','chi_2','chi_spec2']
df_runs[col_list].head(11)